In [13]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import log10 as log10
import math
import helpers
import pandas as pd
from scipy.integrate import quad
import scipy

helpers.figures.initialize()


class PhosphorBronze:
    def thermal_conductivity_NIST(self, T):
        """
        NISTmonograph177-Propertiesofcopperandcopperalloysatcryogenictemperatures--OCR.pdf
        :param T:
        :return:
        """
        tin_concentration = 5  # in percent
        a = 0.4145
        b = 1.563
        c = -0.2285
        d = -0.3234
        e = 0.02500
        return 10 ** (
            a
            + b * np.log10(T)
            - c * np.log10(T) ** 2
            - d * tin_concentration
            + e * tin_concentration**2
        )

    def thermal_conductivity(self, T):
        """
        Data from https://ntrs.nasa.gov/api/citations/20090032058/downloads/20090032058.pdf
        """
        a, b, c, d, e, f, g, h, i = (
            -10.9482,
            28.4752,
            -32.3378,
            20.9036,
            -8.05399,
            1.90329,
            -0.271774,
            0.0215998,
            -7.35095e-4,
        )
        return math.e ** (
            a
            + b * np.log(T)
            + c * np.log(T) ** 2
            + d * np.log(T) ** 3
            + e * np.log(T) ** 4
            + f * np.log(T) ** 5
            + g * np.log(T) ** 6
            + h * np.log(T) ** 7
            + i * np.log(T) ** 8
        )

    def specific_heat(self, T):
        print("____")
        tin_concentration = 5
        debye = lambda x: math.e**x * x**4 / (math.e**x - 1) ** 2

        gamma = (
            0.698
            + 0.00911 * tin_concentration
            - 0.00128 * tin_concentration**2
            + 6.54e-5 * tin_concentration**3
        )
        theta_D = 344 - 3.11 * tin_concentration
        M = 1.187 * tin_concentration + 0.6354 * (100 - tin_concentration)  # gramm/mol

        x_m = theta_D / T
        debye_result, err = quad(debye, 0, x_m)
        debye_value = 7.48e4 * debye_result / (x_m**3)

        if 1.5 < T <= 4:
            return gamma * T / M + 1.94e6 * (T / theta_D) ** 3 / M
        c_i = debye_value / M
        c_e = gamma * T / M
        return c_e + c_i

    def _fit_thermal_expansion(self):
        data = pd.read_csv("cte.csv", index_col="temp")["cte"]
        return scipy.interpolate.interp1d(
            x=data.index, y=data, assume_sorted=True, kind="cubic"
        )

    def thermal_expansion(self, T):
        """
        data from https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=913059
        :param T: temperature in kelvin
        :return: coefficient of thermal expansion in 1/Kelvin
        """
        return self._fit_thermal_expansion()(T) * 1e-6


# src https://trc.nist.gov/cryogenics/materials/Teflon/Teflon_rev.htm


class Teflon:
    def specific_heat(self, T: float) -> float:
        a, b, c, d, e, f, g, h, i = (
            31.88256,
            -166.51949,
            352.01879,
            -393.44232,
            259.98072,
            -104.61429,
            24.99276,
            -3.20792,
            0.16503,
        )
        return 10 ** (
            a
            + b * log10(T)
            + c * (log10(T)) ** 2
            + d * (log10(T)) ** 3
            + e * (log10(T)) ** 4
            + f * (log10(T)) ** 5
            + g * (log10(T)) ** 6
            + h * (log10(T)) ** 7
            + i * (log10(T)) ** 8
        )

    def heat_conductivity(self, T):
        a, b, c, d, e, f, g, h, i = (
            2.7380,
            -30.677,
            89.430,
            -136.99,
            124.69,
            -69.556,
            23.320,
            -4.3135,
            0.33829,
        )
        return 10 ** (
            a
            + b * log10(T)
            + c * (log10(T)) ** 2
            + d * (log10(T)) ** 3
            + e * (log10(T)) ** 4
            + f * (log10(T)) ** 5
            + g * (log10(T)) ** 6
            + h * (log10(T)) ** 7
            + i * (log10(T)) ** 8
        )


teflon = Teflon()
phosphor_bronze = PhosphorBronze()
fig_width, fig_height = helpers.figures.set_size(fraction=1, subplots=(1, 1))
fig, ax1 = plt.subplots(figsize=(fig_width, fig_height * 0.7))

temps = np.logspace(log10(4), log10(300))

heats_teflon = teflon.heat_conductivity(temps)
heats_phosphor_bronze = phosphor_bronze.thermal_conductivity(temps)
ax1.loglog(temps, heats_teflon, label="PTFE")
ax1.loglog(temps, heats_phosphor_bronze, label="Phosphor Bronze")
xlabel = ax1.set_xlabel("Temperature (K)")
ylabel = ax1.set_ylabel("Thermal conductivity (W/m·K)")


ax1.grid(axis="x", which="major")
plt.legend()
fig.savefig(
    "teflon-phbronze-combined.ignore.pdf",
    backend="pgf",
)
fig.savefig(
    "teflon-phbronze-combined.pgf",
    backend="pgf",
)
plt.close(fig)